In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import random
import os

In [5]:
#we want to select some batch data from our stocks folder
vals = []
for i in range(5):
    number = random.randint(0,len(os.listdir("../data/raw/Stock_data/Stocks")))
    vals.append(number)

counter = 0
test_stocks = []
for file in os.listdir("../data/raw/Stock_data/Stocks"):
    if counter in vals:
        df = pd.read_csv("../data/raw/Stock_data/Stocks/" + file)
        test_stocks.append(df)
        print(file)
    counter += 1
    



alo.us.txt
cib.us.txt
pacb.us.txt
petz.us.txt
refr.us.txt
